# Project: Extra Exploration for Kaggle Competition
## Additional Steps: Using The Previous DT Algorithm On Kaggle
In this part, we make the last prediction of this Udacity project competition-ready. We start by getting the test data and than reprocessing it into the last algorithm that had an accuracy of 80.70%. We'll see that this is not a good strategy and we'll explore the data further with different algorithms in order to increase our score.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# RMS Titanic data visualization code 
from titanic_visualizations import survival_stats
from IPython.display import display
%matplotlib inline

# Load the datasets
in_file = 'titanic_data.csv'
full_data = pd.read_csv(in_file)
test = pd.read_csv('test.csv')

# Print the first few entries of the RMS Titanic data
display(full_data.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
outcomes = full_data['Survived']
data = full_data.drop('Survived', axis = 1)

## Extra Exploration
The data exploration that we accomplished in Project 0 of the Nanodegree was mostly visual and we didn't bother processing some of the data i.e. NaN values or transforming categorical data to numeric data. We will accomplish some of these steps here. Some data cleaning needs to be done as well, we'll start with that.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
PassengerId    891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 76.6+ KB


In [6]:
data = data.append(test)
data.reset_index(inplace=True)
data.drop('index',inplace=True,axis=1)

In [7]:
data.shape

(1309, 11)

We see a difference in the count for the Age column (891-714) of 177. This means 177 values are unaccounted for. This certainly is not helping the prediction_4 algorithm. We will need to fill these values because 'Age' is an important feature. Because there are so many missing values for 'Age', using the mean or the median of these values, does not seem to be the best path. The good idea might be to infer the age of people with their titles (found in the 'Name' column).

In [8]:
# For each column processed, we run a short function tell us all went well. 
def status(feature):
    print 'Processed',feature,'column.'

In [9]:
# Extract titles from 'Name' column
def get_titles():
    data['Title'] = data['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
    # Map of titles
    Title_Dictionary = {
                        "Capt":       "Officer",
                        "Col":        "Officer",
                        "Major":      "Officer",
                        "Jonkheer":   "Royalty",
                        "Don":        "Royalty",
                        "Sir" :       "Royalty",
                        "Dr":         "Officer",
                        "Rev":        "Officer",
                        "the Countess":"Royalty",
                        "Dona":       "Royalty",
                        "Mme":        "Mrs",
                        "Mlle":       "Miss",
                        "Ms":         "Mrs",
                        "Mr" :        "Mr",
                        "Mrs" :       "Mrs",
                        "Miss" :      "Miss",
                        "Master" :    "Master",
                        "Lady" :      "Royalty"
                        }
    # Map each title
    data['Title'] = data.Title.map(Title_Dictionary)
    status('Title')

In [10]:
get_titles()

Processed Title column.


In [11]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


We see the added Title column.

In [12]:
data.shape

(1309, 12)

In order to process the 'Age' column, we'll need to explore a little bit more about who is who and what is the median age of each person or group. Let's have a look and group by category.

In [13]:
grouped = data.groupby(['Sex','Pclass','Title'])
grouped.median()

PassengerId   Age  SibSp  Parch      Fare
Sex    Pclass Title                                             
female 1      Miss           529.5  30.0    0.0    0.0   99.9625
              Mrs            853.5  45.0    1.0    0.0   78.1125
              Officer        797.0  49.0    0.0    0.0   25.9292
              Royalty        760.0  39.0    0.0    0.0   86.5000
       2      Miss           606.5  20.0    0.0    0.0   20.2500
              Mrs            533.0  30.0    1.0    0.0   26.0000
       3      Miss           603.5  18.0    0.0    0.0    8.0500
              Mrs            668.5  31.0    1.0    1.0   15.5000
male   1      Master         803.0   6.0    1.0    2.0  134.5000
              Mr             634.0  41.5    0.0    0.0   47.1000
              Officer        678.0  52.0    0.0    0.0   37.5500
              Royalty        600.0  40.0    0.0    0.0   27.7208
       2      Master         550.0   2.0    1.0    1.0   26.0000
              Mr             723.5  30.0    0.0    0.0   13.0000
              Officer        513.0  41.5    0.0    0.0   13.0000
       3      Master         789.0   6.0    3.0    1.0   22.3583
              Mr             640.5  26.0    0.0    0.0    7.8958

Now that we have a clearer picture, we will write a function that fill the missing values with the median of the groups we just made. 

In [14]:
def process_age():
    global data
    # a function that fills the missing values of the Age variable
    def fillAges(row):
        if row['Sex']=='female' and row['Pclass'] == 1:
            if row['Title'] == 'Miss':
                return 30
            elif row['Title'] == 'Mrs':
                return 45
            elif row['Title'] == 'Officer':
                return 49
            elif row['Title'] == 'Royalty':
                return 39

        elif row['Sex']=='female' and row['Pclass'] == 2:
            if row['Title'] == 'Miss':
                return 20
            elif row['Title'] == 'Mrs':
                return 30

        elif row['Sex']=='female' and row['Pclass'] == 3:
            if row['Title'] == 'Miss':
                return 18
            elif row['Title'] == 'Mrs':
                return 31

        elif row['Sex']=='male' and row['Pclass'] == 1:
            if row['Title'] == 'Master':
                return 6
            elif row['Title'] == 'Mr':
                return 41.5
            elif row['Title'] == 'Officer':
                return 52
            elif row['Title'] == 'Royalty':
                return 40

        elif row['Sex']=='male' and row['Pclass'] == 2:
            if row['Title'] == 'Master':
                return 2
            elif row['Title'] == 'Mr':
                return 30
            elif row['Title'] == 'Officer':
                return 41.5

        elif row['Sex']=='male' and row['Pclass'] == 3:
            if row['Title'] == 'Master':
                return 6
            elif row['Title'] == 'Mr':
                return 26
    
    data.Age = data.apply(lambda r : fillAges(r) if np.isnan(r['Age']) else r['Age'], axis=1)
    
    status('Age')

In [15]:
process_age()

Processed Age column.


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1309 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Title          1309 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 122.8+ KB


The Age column is filled, let's turn our attention to the Name column. Now that we have the title, the name does not good for our prediction.

In [17]:
def process_names():
    global data
    # Remove the Name column
    data.drop('Name',axis=1,inplace=True)
    
    # Encode dummy variables
    titles_dummies = pd.get_dummies(data['Title'],prefix='Title')
    data = pd.concat([data,titles_dummies],axis=1)
    
    # Remove the Title column
    data.drop('Title',axis=1,inplace=True)
    
    status('Name')

In [18]:
process_names()

Processed Name column.


The Name column was updated. We see 1 missing value in the Fare column, 2 missing values for the Embarked column and 687 values for the Cabin. We will need to do something about these two columns. But first, let's process the Fare column. 

In [20]:
# Only one value is missing, let's replace the NaN with the mean.
def process_fares():    
    global data
    # Replace NaN with mean of the column
    data.Fare.fillna(data.Fare.mean(),inplace=True)  
    status('Fare')

In [21]:
process_fares()

Processed Fare column.


Let's move on to the Embarked column. We'll use the most common Embarked value to fill the missing values.

In [22]:
data['Embarked'].value_counts()

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [23]:
def process_embarked():
    global data
    # Fill NaN with (S), because it is more prevalent 
    data.Embarked.fillna('S',inplace=True)
    
    # Dummy encoding for modeling later on 
    embarked_dummies = pd.get_dummies(data['Embarked'],prefix='Embarked')
    data = pd.concat([data, embarked_dummies],axis=1)
    data.drop('Embarked',axis=1,inplace=True)
    
    status('Embarked')

In [24]:
process_embarked()

Processed Embarked column.


In [25]:
data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty,Embarked_C,Embarked_Q,Embarked_S
0,1,3,male,22.0,1,0,A/5 21171,7.2500,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,female,38.0,1,0,PC 17599,71.2833,C85,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,3,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,female,35.0,1,0,113803,53.1000,C123,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,5,3,male,35.0,0,0,373450,8.0500,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 18 columns):
PassengerId      1309 non-null int64
Pclass           1309 non-null int64
Sex              1309 non-null object
Age              1309 non-null float64
SibSp            1309 non-null int64
Parch            1309 non-null int64
Ticket           1309 non-null object
Fare             1309 non-null float64
Cabin            295 non-null object
Title_Master     1309 non-null float64
Title_Miss       1309 non-null float64
Title_Mr         1309 non-null float64
Title_Mrs        1309 non-null float64
Title_Officer    1309 non-null float64
Title_Royalty    1309 non-null float64
Embarked_C       1309 non-null float64
Embarked_Q       1309 non-null float64
Embarked_S       1309 non-null float64
dtypes: float64(11), int64(4), object(3)
memory usage: 184.1+ KB


We need to turn our attention to the Cabin column, where the most values are missing. We'll write 'N' for Not Known instead of NaN.

In [27]:
def process_cabin():
    global data
    
    # Replace NaN with N
    data.Cabin.fillna('N',inplace=True)
    
    # Match Cabin value with cabin letter
    data['Cabin'] = data['Cabin'].map(lambda c : c[0])
    
    # Dummy encoding.
    cabin_dummies = pd.get_dummies(data['Cabin'],prefix='Cabin')
    
    data = pd.concat([data,cabin_dummies],axis=1)
    
    data.drop('Cabin',axis=1,inplace=True)
    
    status('Cabin')

In [28]:
process_cabin()

Processed Cabin column.


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 26 columns):
PassengerId      1309 non-null int64
Pclass           1309 non-null int64
Sex              1309 non-null object
Age              1309 non-null float64
SibSp            1309 non-null int64
Parch            1309 non-null int64
Ticket           1309 non-null object
Fare             1309 non-null float64
Title_Master     1309 non-null float64
Title_Miss       1309 non-null float64
Title_Mr         1309 non-null float64
Title_Mrs        1309 non-null float64
Title_Officer    1309 non-null float64
Title_Royalty    1309 non-null float64
Embarked_C       1309 non-null float64
Embarked_Q       1309 non-null float64
Embarked_S       1309 non-null float64
Cabin_A          1309 non-null float64
Cabin_B          1309 non-null float64
Cabin_C          1309 non-null float64
Cabin_D          1309 non-null float64
Cabin_E          1309 non-null float64
Cabin_F          1309 non-null float64
Cabin_

Every missing values has been taken care of. Let's prepare the data for modeling. 

In [30]:
# We take a look at the data first
data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Title_Master,Title_Miss,...,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_N,Cabin_T
0,1,3,male,22.0,1,0,A/5 21171,7.2500,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,1,female,38.0,1,0,PC 17599,71.2833,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,3,female,26.0,0,0,STON/O2. 3101282,7.9250,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,1,female,35.0,1,0,113803,53.1000,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,3,male,35.0,0,0,373450,8.0500,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [31]:
# Function to process the Sex column
def process_sex():
    global data
    # mapping string values to numerical one 
    data['Sex'] = data['Sex'].map({'male':1,'female':0})
    status('Sex')

In [32]:
process_sex()

Processed Sex column.


In [33]:
# Function for the Pclass column
def process_pclass():
    global data
    # Encoding 3 categories:
    pclass_dummies = pd.get_dummies(data['Pclass'],prefix="Pclass")
    # Dummy variables
    data = pd.concat([data,pclass_dummies],axis=1)
    # Remove Pclass column
    data.drop('Pclass',axis=1,inplace=True)
    status('Pclass')

In [34]:
process_pclass()

Processed Pclass column.


In [35]:
# Function to process the Ticket column
def process_ticket():
    
    global data
    # Extract prefix of the ticket, returns 'XXX' if no prefix
    def cleanTicket(ticket):
        ticket = ticket.replace('.','')
        ticket = ticket.replace('/','')
        ticket = ticket.split()
        ticket = map(lambda t : t.strip() , ticket)
        ticket = filter(lambda t : not t.isdigit(), ticket)
        if len(ticket) > 0:
            return ticket[0]
        else: 
            return 'XXX'
    
    # Dummy variables:
    data['Ticket'] = data['Ticket'].map(cleanTicket)
    tickets_dummies = pd.get_dummies(data['Ticket'],prefix='Ticket')
    data = pd.concat([data, tickets_dummies],axis=1)
    data.drop('Ticket',inplace=True,axis=1)

    status('Ticket')

In [36]:
process_ticket()

Processed Ticket column.


In [37]:
# To process the family, we need to create that variable from the Parch and SibSp columns
def process_family():
    
    global data
    # New feature : the size of families (including the passenger)
    data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
    
    # Introduce other features based on the family size
    data['Family_of_One'] = data['FamilySize'].map(lambda s : 1 if s == 1 else 0)
    data['SmallFamily'] = data['FamilySize'].map(lambda s : 1 if 2<=s<=4 else 0)
    data['LargeFamily'] = data['FamilySize'].map(lambda s : 1 if 5<=s else 0)
    
    status('Family')

In [38]:
process_family()

Processed Family column.


In [39]:
# Normalize all features
def scale_all_features():
    global data
    features = list(data.columns)
    features.remove('PassengerId')
    data[features] = data[features].apply(lambda x: x/x.max(), axis=0)
    
    print 'Features scaled successfully !'

In [40]:
scale_all_features()

Features scaled successfully !


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 68 columns):
PassengerId       1309 non-null int64
Sex               1309 non-null float64
Age               1309 non-null float64
SibSp             1309 non-null float64
Parch             1309 non-null float64
Fare              1309 non-null float64
Title_Master      1309 non-null float64
Title_Miss        1309 non-null float64
Title_Mr          1309 non-null float64
Title_Mrs         1309 non-null float64
Title_Officer     1309 non-null float64
Title_Royalty     1309 non-null float64
Embarked_C        1309 non-null float64
Embarked_Q        1309 non-null float64
Embarked_S        1309 non-null float64
Cabin_A           1309 non-null float64
Cabin_B           1309 non-null float64
Cabin_C           1309 non-null float64
Cabin_D           1309 non-null float64
Cabin_E           1309 non-null float64
Cabin_F           1309 non-null float64
Cabin_G           1309 non-null float64
Cabin_N        

We are done with feature engineering for now. 

## Modeling For Different Algorithms
We'll try the Random Forest algorithm to start off. We'll import the necessary libraries and make the training and testing sets. 

In [43]:
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV

To evaluate our model we'll be using 5-fold cross validation with the Accuracy metric.

In [44]:
def compute_score(clf, X, y,scoring='accuracy'):
    xval = cross_val_score(clf, X, y, cv = 5,scoring=scoring)
    return np.mean(xval)

We need to make the train, test and targets variable so we can fit models on them. 

In [46]:
def recover_train_test_target():
    global data
    targets = outcomes
    train = data.ix[0:890]
    test = data.ix[891:1309]
    
    return train,test,targets

In [47]:
train,test,targets = recover_train_test_target()

In [48]:
train.shape

(891, 68)

In [49]:
test.shape

(418, 68)

We make copies of the files, in order to use them in a different notebook to test XGBoost.

In [51]:
test_process = pd.DataFrame()
test_process = test
test_process.to_csv('processed_test.csv',index=False)

In [52]:
train_process = pd.DataFrame()
train_process = train
train_process.to_csv('processed_train.csv',index=False)

In [53]:
target_process = pd.DataFrame()
target_process = targets
target_process.to_csv('processed_target.csv',index=False)

### Feature selection
We have 68 features from the feature engineering that we did previously. We'll want to decrease the dimensionality in order to use only useful features. By selecting useful features, we'll speed up the training process.

In [54]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(n_estimators=200)
clf = clf.fit(train, targets)

In [55]:
features = pd.DataFrame()
features['feature'] = train.columns
features['importance'] = clf.feature_importances_

In [56]:
features.sort_values(['importance'],ascending=False)

,feature,importance
0,PassengerId,0.128331
2,Age,0.119703
8,Title_Mr,0.119244
5,Fare,0.112263
1,Sex,0.110692
26,Pclass_3,0.038494
9,Title_Mrs,0.036133
7,Title_Miss,0.032759
22,Cabin_N,0.027113
24,Pclass_1,0.022009


In [57]:
model = SelectFromModel(clf, prefit=True)
train_new = model.transform(train)
train_new.shape

(891, 15)

In [58]:
test_new = model.transform(test)
test_new.shape

(418, 15)

## Random Forest

In [59]:
forest = RandomForestClassifier(max_features='sqrt', class_weight={0:0.745,1:0.255})

parameter_grid = {
                 'max_depth' : [4,5,6,7,8],
                 'n_estimators': [200,210,240,250],
                 'criterion': ['gini','entropy'],
                 }

cross_validation = StratifiedKFold(targets, n_folds=5)

grid_search = GridSearchCV(forest,
                           param_grid=parameter_grid,
                           cv=cross_validation)

grid_search.fit(train_new, targets)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: 0.817059483726
Best parameters: {'n_estimators': 210, 'criterion': 'gini', 'max_depth': 7}


In [61]:
output = grid_search.predict(test_new).astype(int)
rf_score = pd.DataFrame()
rf_score['PassengerId'] = test['PassengerId']
rf_score['Survived'] = output
rf_score[['PassengerId','Survived']].to_csv('rf_score_dec9.csv',index=False)

Scored 0.78947 on december 9 2016.

## Gradient Boost
Let's try Gradient Boost. Some parameters tuning could be done more in depth. 

In [62]:
from sklearn.ensemble import GradientBoostingClassifier

In [63]:
gradient_boost = GradientBoostingClassifier(max_features='sqrt', random_state=10)

parameter_grid = {
                 'min_samples_leaf' : range(2, 10, 1),
                 'max_depth' : range(2, 7, 1),
                 'n_estimators': [400,500,600]
                 }

cross_validation = StratifiedKFold(targets, n_folds=5)

grid_search = GridSearchCV(gradient_boost,
                           param_grid = parameter_grid,
                           cv=cross_validation)

grid_search.fit(train_new, targets)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: 0.806958473625
Best parameters: {'n_estimators': 400, 'max_depth': 5, 'min_samples_leaf': 2}


In [65]:
output = grid_search.predict(test_new).astype(int)
gb_score = pd.DataFrame()
gb_score['PassengerId'] = test['PassengerId']
gb_score['Survived'] = output
gb_score[['PassengerId','Survived']].to_csv('gb_score_dec9.csv',index=False)

Scored 0.76555 on december 9 2016.